# **Multi-Precision CNN**
*   Implementation of a convolutional nerural network supporting multi-precision training.
*   Implementation currently supports training in either double, single, or half precision.
*   Implementation is based off of TensorFlow's very own example: [TensorFlow CNN Tutorial](https://www.tensorflow.org/tutorials/images/cnn)







In [16]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [17]:
# Define number of training runs to compute the average training time over
NUM_TRAINING_RUNS = 2
# Values are specific to the CIFAR10 dataset
INPUT_DIM = 32
INPUT_CHANNELS = 3
NUM_CLASSES = 10

In [18]:
def build_and_train(X_train, y_train, precision='single'):
    if precision == 'double':
        dtype = tf.float64
    elif precision == 'single':
        dtype = tf.float32
    else: # half
        dtype = tf.float16

    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(INPUT_DIM, INPUT_DIM, INPUT_CHANNELS), dtype=dtype),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', dtype=dtype),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(dtype=dtype),
        layers.Dense(64, activation='relu', dtype=dtype),
        layers.Dense(NUM_CLASSES, activation='softmax', dtype=dtype)
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    start_time = time.time()
    model.fit(X_train, y_train, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    return model, training_time

In [19]:
# Load dataset and split into train and test sets
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()

# Scale values before feeding into neural net
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
# Test run to make sure that everything is working properly before starting actual measurements
_ = build_and_train(X_train, y_train, precision='single')

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1563/1563 [==============================] - 13s 5ms/step - loss: 1.5162 - accuracy: 0.4464
Epoch 2/5
1563/1563 [==============================] - 7s 4ms/step - loss: 1.1457 - accuracy: 0.5948
Epoch 3/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.0007 - accuracy: 0.6492
Epoch 4/5
1563/1563 [==============================] - 9s 5ms/step - loss: 0.9095 - accuracy: 0.6819
Epoch 5/5
1563/1563 [==============================] - 7s 5ms/step - loss: 0.8391 - accuracy: 0.7049


In [20]:
# Train with double precision
time_double = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_double, training_time = build_and_train(X_train, y_train, 'double')
    time_double += training_time
accuracy_double = model_double.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1563/1563 [==============================] - 17s 9ms/step - loss: 1.5214 - accuracy: 0.4460
Epoch 2/5
1563/1563 [==============================] - 14s 9ms/step - loss: 1.1606 - accuracy: 0.5879
Epoch 3/5
1563/1563 [==============================] - 14s 9ms/step - loss: 1.0148 - accuracy: 0.6434
Epoch 4/5
1563/1563 [==============================] - 14s 9ms/step - loss: 0.9291 - accuracy: 0.6729
Epoch 5/5
1563/1563 [==============================] - 13s 9ms/step - loss: 0.8600 - accuracy: 0.6988
Epoch 1/5
1563/1563 [==============================] - 16s 9ms/step - loss: 1.5167 - accuracy: 0.4474
Epoch 2/5
1563/1563 [==============================] - 14s 9ms/step - loss: 1.1480 - accuracy: 0.5939
Epoch 3/5
1563/1563 [==============================] - 14s 9ms/step - loss: 1.0006 - accuracy: 0.6508
Epoch 4/5
1563/1563 [==============================] - 14s 9ms/step - loss: 0.9015 - accuracy: 0.6836
Epoch 5/5
1563/1563 [==============================] - 14s 9ms/step - loss: 0.8362

In [21]:
# Train with single precision
time_single = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_single, training_time = build_and_train(X_train, y_train, 'single')
    time_single += training_time
accuracy_single = model_single.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1563/1563 [==============================] - 9s 5ms/step - loss: 1.5045 - accuracy: 0.4539
Epoch 2/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.1497 - accuracy: 0.5914
Epoch 3/5
1563/1563 [==============================] - 7s 5ms/step - loss: 1.0069 - accuracy: 0.6455
Epoch 4/5
1563/1563 [==============================] - 8s 5ms/step - loss: 0.9121 - accuracy: 0.6790
Epoch 5/5
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8416 - accuracy: 0.7048
Epoch 1/5
1563/1563 [==============================] - 9s 5ms/step - loss: 1.5071 - accuracy: 0.4521
Epoch 2/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.1296 - accuracy: 0.6003
Epoch 3/5
1563/1563 [==============================] - 8s 5ms/step - loss: 0.9855 - accuracy: 0.6529
Epoch 4/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.8890 - accuracy: 0.6880
Epoch 5/5
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8194 - accurac

In [22]:
# Train with half precision
time_half = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_half, training_time = build_and_train(X_train, y_train, 'half')
    time_half += training_time
accuracy_half = model_half.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1563/1563 [==============================] - 9s 5ms/step - loss: nan - accuracy: 0.0999
Epoch 2/5
1563/1563 [==============================] - 7s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 3/5
1563/1563 [==============================] - 8s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 4/5
1563/1563 [==============================] - 7s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 5/5
1563/1563 [==============================] - 8s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 1/5
1563/1563 [==============================] - 9s 5ms/step - loss: nan - accuracy: 0.0999
Epoch 2/5
1563/1563 [==============================] - 8s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 3/5
1563/1563 [==============================] - 7s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 4/5
1563/1563 [==============================] - 8s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 5/5
1563/1563 [==============================] - 7s 5ms/step - loss: nan - accuracy: 0.1000
313/313 - 1s - loss:

In [23]:
print("---RESULTS---")
print("Average training time in double precision:", time_double / NUM_TRAINING_RUNS, "seconds")
print("Average training time in single precision:", time_single/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in half   precision:", time_half/ NUM_TRAINING_RUNS, "seconds")
print("-------------")
print("Accuracy with double precision:", accuracy_double)
print("Accuracy with single precision:", accuracy_single)
print("Accuracy with half   precision:", accuracy_single)

---RESULTS---
Average training time in double precision: 72.99917113780975 seconds
Average training time in single precision: 43.175623059272766 seconds
Average training time in half   precision: 42.78127717971802 seconds
-------------
Accuracy with double precision: 0.6854000091552734
Accuracy with single precision: 0.6934000253677368
Accuracy with half   precision: 0.6934000253677368
